# Tutorial BCI Datos ADLS y Create Database bajo nuevo estandard

## Datos en Storage ADLS v2

La data estara en un Azure DataLake Storage (ADLS)

### Instalar File Browser Azure Storage Explorer

este es un utilitario que sirve para manejar archivos en ADLS

Instalar en desktop local

https://azure.microsoft.com/en-us/features/storage-explorer/

Conectarse al Storage de la POC con opción "Open Connect" (icono enchufe), seleccionar "use a Connection string"

En campo "Connection String" copiar string de conexion de Azure Storage account 
  
En arbol se va a ubicar nuestro storage bajo:

- Local & Attaches
- - Storage Accounts

Aqui pueden usarlo como File Browser y subir archivos de datos

### Conectarse desde este Notebook (DataSci)

vamos a montar con protocolo ADLSv2 que permite  escritura

In [0]:
%fs ls /mnt/bronze

Vamos a crear Base de Datos apuntando ADLS

ref https://docs.databricks.com/spark/latest/spark-sql/language-manual/create-database.html

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS bdrcred_test_db LOCATION '/mnt/bronze/bdrcred_test_db/';

se creo carpeta de bdatos en ADLS

In [0]:
%fs ls /mnt/bronze

Ahora toda tabla que creemos en esta base de datos va a estar en ADLS por defecto

In [0]:
%sql
CREATE TABLE bdrcred_test_db.fraudetest_tb (`id_cliente` STRING, `jen_fec_evt_neg` STRING, `jen_num_ope_evt` STRING, `jen_id_cnl` STRING, `jen_id_pdt` STRING, `jen_id_evt` STRING, `jen_id_sub_evt` STRING, `jen_med` STRING, `target` STRING, `inicio` STRING) 
USING parquet

In [0]:
%sql
show create table bdrcred_test_db.fraudetest_tb

se creo carpeta de la tabla en ADLS

In [0]:
%fs ls /mnt/bronze/bdrcred_test_db

In [0]:
%sql

INSERT INTO bdrcred_test_db.fraudetest_tb VALUES (
'4363799',
'2019-03-28 15:16:01.000000',
'WW0929608943',
'MOVIBCINAT',
'INT',
'LOGIN',
'OK',
'10.10.10.2',
'0',
'2019-03-28 15:23:26.00000');

In [0]:
%sql
REFRESH TABLE bdrcred_test_db.fraudetest_tb

In [0]:
%sql
select * from bdrcred_test_db.fraudetest_tb where id_cliente = '4363799'

In [0]:
%fs ls /mnt/bronze/bdrcred_test_db/fraudetest_tb

Tambien podemos listarlo desde Python con dbutils

In [0]:
display(dbutils.fs.ls("dbfs:/mnt/bronze/bdrcred_test_db/fraudetest_tb"))

y ya podemos realizar dataframes usando spark

In [0]:
dfparquet = spark.read.parquet("/mnt/bronze/bdrcred_test_db/fraudetest_tb")

In [0]:
dfparquet.show()